In [161]:
import xgboost as xgb
import pandas as pd
import numpy as np
from datetime import timedelta,datetime
import dataframe_image as dfi
import matplotlib.pyplot as plt
import sklearn.metrics as mtr
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import pickle


# Primeira Execução

In [162]:
rota = 33642
df = pd.read_csv('rota33642.csv')
df.head()

,data_partida,data_chegada,linha,tempo_viagem,equipamento,dia_semana,partidaTimeStamp,qtdDiasAno,direcao,tipo_dia,turno_dia,chegadaTimeStamp,hora,hora_dia
0,2019-07-02 05:35:51,2019-07-02 05:47:44,33642,713,1083,2,1562056551,183,2,1,2,1562057264,NaN,5
1,2019-07-02 06:11:07,2019-07-02 06:25:15,33642,848,1083,2,1562058667,183,2,1,2,1562059515,NaN,6
2,2019-07-02 06:53:26,2019-07-02 07:08:02,33642,876,1083,2,1562061206,183,2,1,2,1562062082,NaN,6
3,2019-07-02 07:36:44,2019-07-02 07:55:03,33642,1099,1083,2,1562063804,183,2,1,2,1562064903,NaN,7
4,2019-07-02 08:16:37,2019-07-02 08:30:55,33642,858,1083,2,1562066197,183,2,1,2,1562067055,NaN,8


In [163]:
# Dropamos:
# A data_chegada e a chegadaTimeStamp pois com esses valores daria para prever o tempo de viagem(Y)
# linha e equipamento por se tratar apenas de IDs
# hora por conter apenas valores nulos
# direcao por conter apenas o valor 2

df = df.drop(['data_chegada','linha','equipamento','direcao','chegadaTimeStamp','hora'], axis=1)

In [164]:
corr = df.corr()
corr

C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\2438084875.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = df.corr()


,tempo_viagem,dia_semana,partidaTimeStamp,qtdDiasAno,tipo_dia,turno_dia,hora_dia
tempo_viagem,1.000000,-0.091237,0.036642,0.033283,-0.041862,0.360651,0.375852
dia_semana,-0.091237,1.000000,0.018559,0.018371,0.044735,0.018066,0.020579
partidaTimeStamp,0.036642,0.018559,1.000000,0.999960,-0.012245,0.006308,0.005832
qtdDiasAno,0.033283,0.018371,0.999960,1.000000,-0.012287,-0.002152,-0.003087
tipo_dia,-0.041862,0.044735,-0.012245,-0.012287,1.000000,0.001882,0.004642
turno_dia,0.360651,0.018066,0.006308,-0.002152,0.001882,1.000000,0.948907
hora_dia,0.375852,0.020579,0.005832,-0.003087,0.004642,0.948907,1.000000


In [165]:
df[['hora_dia','turno_dia']].corr()

,hora_dia,turno_dia
hora_dia,1.000000,0.948907
turno_dia,0.948907,1.000000


In [166]:
df[['partidaTimeStamp','qtdDiasAno']].corr()

,partidaTimeStamp,qtdDiasAno
partidaTimeStamp,1.00000,0.99996
qtdDiasAno,0.99996,1.00000


In [167]:
#Vamos tirar hora_dia pois tem uma alta correlação com turno dia, escolhemos essa coluna pois os valores são apenas de 1 a 4 enquanto hora é de 0 a 23
df = df.drop(['qtdDiasAno'], axis=1)

In [168]:
#Ordenando pela Data
df = df.sort_values(by=["data_partida"], ascending=False)

In [169]:
#Separamos treino e teste
shape = int(df.shape[0] * 0.3)
df_teste = df[:shape]
df_treino = df[shape:]
print(f'Treino: {df_treino.shape}      Teste:{df_teste.shape}')

Treino: (6520, 7)      Teste:(2794, 7)


In [170]:
# Separamos a classe dos atributos 
x_teste = df_teste[['dia_semana','partidaTimeStamp','tipo_dia','hora_dia']]
y_teste = df_teste[['tempo_viagem']]
x_treino = df_treino[['dia_semana','partidaTimeStamp','tipo_dia','hora_dia']]
y_treino = df_treino[['tempo_viagem']]

In [171]:
params_xgb = {
        "n_estimators": list(range(100, 1100, 100)), #Number of gradient boosted trees. Equivalent to number of boosting rounds
        "max_depth": list(range(2, 15)),#Maximum tree depth for base learners.
        "min_child_weight": list(range(1, 11)),#Minimum sum of instance weight(hessian) needed in a child.
        "learning_rate": [0.3, 0.2, 0.1, 0.05, 0.01, 0.005],#Boosting learning rate (xgb’s “eta”)
        "gamma": np.arange(0, 0.7, 0.1)#Minimum loss reduction required to make a further partition on a leaf node of the tree.
        }

# Hyperparâmetros do Random Forest
params_rf = {
        "n_estimators": list(range(100, 1100, 100)),#The number of trees in the forest.
        "bootstrap": [True, False],#Whether bootstrap samples are used when building trees. If False, the whole dataset is used to build each tree.
        "max_depth": list(range(2, 15)),#The maximum depth of the tree.
        "max_features": [1.0, "sqrt", "log2"],#The number of features to consider when looking for the best split:
        "min_samples_leaf": list(range(1, 11)),#The minimum number of samples required to split an internal node
        "min_samples_split": list(range(2, 11)),#The minimum number of samples required to be at a leaf node.
        }

In [172]:
# xgb_rand_search.best_estimator_

In [173]:
# rf_rand_search.best_estimator_

In [174]:
#Criação e fit do modelo XGB
start_xgb = datetime.now()
modelo_xgb = xgb.XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stop_rounds=100, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=0.0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.005, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=3, max_leaves=0,
             min_child_weight=5, monotone_constraints='()',
             n_estimators=900, n_jobs=0, num_parallel_tree=1, predictor='auto',
             random_state=0, reg_alpha=0)
#modelo_xgb = xgb.XGBRegressor(early_stop_rounds = 100)
#xgb_rand_search = RandomizedSearchCV(modelo_xgb, params_xgb, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
#xgb_rand_search.fit(x_treino, y_treino)
#modelo_xgb = xgb_rand_search.best_estimator_
modelo_xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino)],verbose=False)
end_xgb = datetime.now()

#Criação e fit do modelo Random Forest
start_rf = datetime.now()
modelo_rf = RandomForestRegressor(max_depth=10, max_features='log2', min_samples_leaf=4,
                      min_samples_split=7, n_estimators=900)
#modelo_rf = RandomForestRegressor()
#rf_rand_search = RandomizedSearchCV(modelo_rf, params_rf, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
#rf_rand_search.fit(x_treino, y_treino)
#modelo_rf = rf_rand_search.best_estimator_
modelo_rf.fit(x_treino, y_treino)
end_rf = datetime.now()

#Criação e fit do modelo Linear Regression
start_lr = datetime.now()
modelo_lr = LinearRegression()
modelo_lr.fit(x_treino, y_treino)
end_lr = datetime.now()


tempo_xgb = end_xgb - start_xgb
tempo_xgb = float(f"{str(tempo_xgb.seconds)}.{str(tempo_xgb.microseconds)}")
tempo_rf = end_rf - start_rf
tempo_rf = float(f"{str(tempo_rf.seconds)}.{str(tempo_rf.microseconds)}")
tempo_lr = end_lr - start_lr
tempo_lr = float(f"{str(tempo_lr.seconds)}.{str(tempo_lr.microseconds)}")

[21:46:01] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\995122566.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  modelo_rf.fit(x_treino, y_treino)


In [175]:
# Fizemos as predições e juntamos no DataFrame
df_teste["predicao_xgb"] = modelo_xgb.predict(x_teste)
df_teste["predicao_lr"] = modelo_lr.predict(x_teste)
df_teste["predicao_rf"] = modelo_rf.predict(x_teste)

C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\1876206521.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_xgb"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\1876206521.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_lr"] = modelo_lr.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\1876206521.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

In [176]:
# Pesquisar os motivos de se usar apenas métricas relativas - Flávia disse que provavelmente por conta de outlier/valor distantes 
RRSE_xgb = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste["predicao_xgb"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
MAPE_xgb = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste["predicao_xgb"])

RRSE_rf = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste["predicao_rf"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
MAPE_rf = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste["predicao_rf"])

RRSE_lr = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste["predicao_lr"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
MAPE_lr = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste["predicao_lr"])

In [177]:
metrics = []

In [178]:
metrics.append(['Primeira execução','RRSE', RRSE_xgb, RRSE_rf, RRSE_lr])
metrics.append(['Primeira execução','MAPE', MAPE_xgb, MAPE_rf, MAPE_lr]) 
metrics.append(['Primeira execução','Tempo', tempo_xgb, tempo_rf, tempo_lr]) 
metrics

[['Primeira execução',
  'RRSE',
  0.6929011501881002,
  0.7232911739283211,
  0.9394448227378863],
 ['Primeira execução',
  'MAPE',
  0.10364107666716442,
  0.10942823282525131,
  0.15358537448478704],
 ['Primeira execução', 'Tempo', 1.482996, 2.731269, 0.3]]

In [179]:
df = df.merge(df_teste[['predicao_xgb', 'predicao_lr', 'predicao_rf']], how="left", left_index=True, right_index=True)
df_teste.head()

,data_partida,tempo_viagem,dia_semana,partidaTimeStamp,tipo_dia,turno_dia,hora_dia,predicao_xgb,predicao_lr,predicao_rf
7351,2019-10-01 00:40:28,680,2,1569901228,1,1,0,831.875977,978.708272,836.503492
6966,2019-10-01 00:16:27,849,2,1569899787,1,1,0,831.875977,978.703918,836.503492
7350,2019-10-01 00:00:45,852,2,1569898845,1,1,0,831.875977,978.701072,836.503492
8004,2019-09-30 23:46:49,824,1,1569898009,1,4,23,1042.120117,1373.619877,1128.561631
6965,2019-09-30 23:30:14,993,1,1569897014,1,4,23,1042.120117,1373.616871,1128.561631


# Teste com Clima

In [180]:
#Lendo dados de clima e convertendo a data pra datetime (na leitura veio como objeto)
clima = pd.read_csv('clima.csv')
clima['Datetime'] = pd.to_datetime(clima['Datetime'])

In [181]:
#copiando o DAtaframe original para não alterar o primeiro
aux = df.copy()
#Mudanças na data_partida para conseguir fazer o merge
aux["data_partida2"] = pd.to_datetime(aux["data_partida"])
aux["data_partida2"] = aux["data_partida2"].dt.round("H")
#Merge dos dados e exclusão da coluna auxiliar
df_clima = aux.merge(clima, left_on="data_partida2", right_on="Datetime", how="left")
df_clima.drop(['data_partida2','Datetime'],axis=1,inplace=True)
df_clima.index = aux.index
df_clima.head()

,data_partida,tempo_viagem,dia_semana,partidaTimeStamp,tipo_dia,turno_dia,hora_dia,predicao_xgb,predicao_lr,predicao_rf,Descricao Chuva,Calor
7351,2019-10-01 00:40:28,680,2,1569901228,1,1,0,831.875977,978.708272,836.503492,0,1
6966,2019-10-01 00:16:27,849,2,1569899787,1,1,0,831.875977,978.703918,836.503492,0,1
7350,2019-10-01 00:00:45,852,2,1569898845,1,1,0,831.875977,978.701072,836.503492,0,1
8004,2019-09-30 23:46:49,824,1,1569898009,1,4,23,1042.120117,1373.619877,1128.561631,0,1
6965,2019-09-30 23:30:14,993,1,1569897014,1,4,23,1042.120117,1373.616871,1128.561631,0,1


In [182]:
#Separamos treino e teste
shape = int(df_clima.shape[0] * 0.3)
df_teste = df_clima[:shape]
df_treino = df_clima[shape:]
print(f'Treino: {df_treino.shape}      Teste:{df_teste.shape}')

Treino: (6520, 12)      Teste:(2794, 12)


In [183]:
# Separamos a classe dos atributos 
x_teste = df_teste[['dia_semana','partidaTimeStamp','tipo_dia','hora_dia','Descricao Chuva','Calor']]
y_teste = df_teste[['tempo_viagem']]
x_treino = df_treino[['dia_semana','partidaTimeStamp','tipo_dia','hora_dia','Descricao Chuva','Calor']]
y_treino = df_treino[['tempo_viagem']]

In [184]:
#Criação e fit do modelo XGB
start_xgb = datetime.now()
modelo_xgb = xgb.XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stop_rounds=100, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=0.0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.005, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=3, max_leaves=0,
             min_child_weight=5, monotone_constraints='()',
             n_estimators=900, n_jobs=0, num_parallel_tree=1, predictor='auto',
             random_state=0, reg_alpha=0)
#modelo_xgb = xgb.XGBRegressor(early_stop_rounds = 100)
#xgb_rand_search = RandomizedSearchCV(modelo_xgb, params_xgb, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
#xgb_rand_search.fit(x_treino, y_treino)
#modelo_xgb = xgb_rand_search.best_estimator_
modelo_xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino)],verbose=False)
end_xgb = datetime.now()

#Criação e fit do modelo Random Forest
start_rf = datetime.now()
modelo_rf = RandomForestRegressor(max_depth=10, max_features='log2', min_samples_leaf=4,
                      min_samples_split=7, n_estimators=900)
#modelo_rf = RandomForestRegressor()
#rf_rand_search = RandomizedSearchCV(modelo_rf, params_rf, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
#rf_rand_search.fit(x_treino, y_treino)
#modelo_rf = rf_rand_search.best_estimator_
modelo_rf.fit(x_treino, y_treino)
end_rf = datetime.now()

#Criação e fit do modelo Linear Regression
start_lr = datetime.now()
modelo_lr = LinearRegression()
modelo_lr.fit(x_treino, y_treino)
end_lr = datetime.now()


tempo_xgb = end_xgb - start_xgb
tempo_xgb = float(f"{str(tempo_xgb.seconds)}.{str(tempo_xgb.microseconds)}")
tempo_rf = end_rf - start_rf
tempo_rf = float(f"{str(tempo_rf.seconds)}.{str(tempo_rf.microseconds)}")
tempo_lr = end_lr - start_lr
tempo_lr = float(f"{str(tempo_lr.seconds)}.{str(tempo_lr.microseconds)}")

[21:46:06] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\995122566.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  modelo_rf.fit(x_treino, y_treino)


In [185]:
# Fizemos as predições e juntamos no DataFrame
df_teste["predicao_xgb_clima"] = modelo_xgb.predict(x_teste)
df_teste["predicao_lr_clima"] = modelo_lr.predict(x_teste)
df_teste["predicao_rf_clima"] = modelo_rf.predict(x_teste)

C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\1375954795.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_xgb_clima"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\1375954795.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_lr_clima"] = modelo_lr.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\1375954795.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [186]:
# Pesquisar os motivos de se usar apenas métricas relativas - Flávia disse que provavelmente por conta de outlier/valor distantes 
RRSE_xgb = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste["predicao_xgb_clima"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
MAPE_xgb = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste["predicao_xgb_clima"])

RRSE_rf = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste["predicao_rf_clima"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
MAPE_rf = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste["predicao_rf_clima"])

RRSE_lr = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste["predicao_lr_clima"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
MAPE_lr = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste["predicao_lr_clima"])

In [187]:

metrics.append(['Teste Clima','RRSE', RRSE_xgb, RRSE_rf, RRSE_lr])
metrics.append(['Teste Clima','MAPE', MAPE_xgb, MAPE_rf, MAPE_lr]) 
metrics.append(['Teste Clima','Tempo', tempo_xgb, tempo_rf, tempo_lr]) 
metrics

[['Primeira execução',
  'RRSE',
  0.6929011501881002,
  0.7232911739283211,
  0.9394448227378863],
 ['Primeira execução',
  'MAPE',
  0.10364107666716442,
  0.10942823282525131,
  0.15358537448478704],
 ['Primeira execução', 'Tempo', 1.482996, 2.731269, 0.3],
 ['Teste Clima',
  'RRSE',
  0.6943001942678831,
  0.7225688289648855,
  0.9160853504082752],
 ['Teste Clima',
  'MAPE',
  0.1031040485254041,
  0.10939611924780253,
  0.1519430606843786],
 ['Teste Clima', 'Tempo', 1.413068, 2.359836, 0.2999]]

In [188]:
df = df.merge(df_teste[['Descricao Chuva','Calor','predicao_xgb_clima', 'predicao_lr_clima', 'predicao_rf_clima']], how="left", left_index=True, right_index=True)
df_teste.head()

,data_partida,tempo_viagem,dia_semana,partidaTimeStamp,tipo_dia,turno_dia,hora_dia,predicao_xgb,predicao_lr,predicao_rf,Descricao Chuva,Calor,predicao_xgb_clima,predicao_lr_clima,predicao_rf_clima
7351,2019-10-01 00:40:28,680,2,1569901228,1,1,0,831.875977,978.708272,836.503492,0,1,841.116333,1026.636094,886.634471
6966,2019-10-01 00:16:27,849,2,1569899787,1,1,0,831.875977,978.703918,836.503492,0,1,841.116333,1026.633915,886.634471
7350,2019-10-01 00:00:45,852,2,1569898845,1,1,0,831.875977,978.701072,836.503492,0,1,841.116333,1026.632491,886.634471
8004,2019-09-30 23:46:49,824,1,1569898009,1,4,23,1042.120117,1373.619877,1128.561631,0,1,1021.690552,1371.491567,1076.737406
6965,2019-09-30 23:30:14,993,1,1569897014,1,4,23,1042.120117,1373.616871,1128.561631,0,1,1021.690552,1371.490062,1076.737406


# Tempos Anteriores

In [189]:
df_tempos = df.copy()
df_tempos["tempo_viagem_1"] = df_tempos["tempo_viagem"]
df_tempos["tempo_viagem_2"] = df_tempos["tempo_viagem"]
df_tempos["tempo_viagem_3"] = df_tempos["tempo_viagem"]
df_tempos["tempo_viagem_4"] = df_tempos["tempo_viagem"]

df_tempos["tempo_viagem_1"] = df_tempos["tempo_viagem_1"].shift(1)
df_tempos["tempo_viagem_2"] = df_tempos["tempo_viagem_2"].shift(2)
df_tempos["tempo_viagem_3"] = df_tempos["tempo_viagem_3"].shift(3)
df_tempos["tempo_viagem_4"] = df_tempos["tempo_viagem_4"].shift(4)
df_tempos.head()

,data_partida,tempo_viagem,dia_semana,partidaTimeStamp,tipo_dia,turno_dia,hora_dia,predicao_xgb,predicao_lr,predicao_rf,Descricao Chuva,Calor,predicao_xgb_clima,predicao_lr_clima,predicao_rf_clima,tempo_viagem_1,tempo_viagem_2,tempo_viagem_3,tempo_viagem_4
7351,2019-10-01 00:40:28,680,2,1569901228,1,1,0,831.875977,978.708272,836.503492,0.0,1.0,841.116333,1026.636094,886.634471,NaN,NaN,NaN,NaN
6966,2019-10-01 00:16:27,849,2,1569899787,1,1,0,831.875977,978.703918,836.503492,0.0,1.0,841.116333,1026.633915,886.634471,680.0,NaN,NaN,NaN
7350,2019-10-01 00:00:45,852,2,1569898845,1,1,0,831.875977,978.701072,836.503492,0.0,1.0,841.116333,1026.632491,886.634471,849.0,680.0,NaN,NaN
8004,2019-09-30 23:46:49,824,1,1569898009,1,4,23,1042.120117,1373.619877,1128.561631,0.0,1.0,1021.690552,1371.491567,1076.737406,852.0,849.0,680.0,NaN
6965,2019-09-30 23:30:14,993,1,1569897014,1,4,23,1042.120117,1373.616871,1128.561631,0.0,1.0,1021.690552,1371.490062,1076.737406,824.0,852.0,849.0,680.0


In [190]:
shape = int(df_tempos.shape[0] * 0.3)
df_teste = df_tempos[:shape]
df_treino = df_tempos[shape:]
print(f'Treino: {df_treino.shape}      Teste:{df_teste.shape}')

Treino: (6520, 19)      Teste:(2794, 19)


In [191]:

for cenario in range(1, 5):
        if cenario == 1:
                df_teste = df_teste.iloc[1:]
                df_treino = df_treino.iloc[1:]

                x_teste = df_teste[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1"]]
                y_teste = df_teste["tempo_viagem"]
                x_treino = df_treino[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1"]]
                y_treino = df_treino["tempo_viagem"]

        elif cenario == 2:
                df_teste = df_teste.iloc[2:]
                df_treino = df_treino.iloc[2:]

                x_teste = df_teste[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"]]
                y_teste = df_teste["tempo_viagem"]
                x_treino = df_treino[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"]]
                y_treino = df_treino["tempo_viagem"]

        elif cenario == 3:
                df_teste = df_teste.iloc[3:]
                df_treino = df_treino.iloc[3:]

                x_teste = df_teste[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3"]]
                y_teste = df_teste["tempo_viagem"]
                x_treino = df_treino[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3"]]
                y_treino = df_treino["tempo_viagem"]
        else:
                df_teste = df_teste.iloc[4:]
                df_treino = df_treino.iloc[4:]

                x_teste = df_teste[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3","tempo_viagem_4"]]
                y_teste = df_teste["tempo_viagem"]
                x_treino = df_treino[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3","tempo_viagem_4"]]
                y_treino = df_treino["tempo_viagem"]

        #Criação e fit do modelo XGB
        start_xgb = datetime.now()
        modelo_xgb = xgb.XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
                colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
                early_stop_rounds=100, early_stopping_rounds=None,
                enable_categorical=False, eval_metric=None, gamma=0.0, gpu_id=-1,
                grow_policy='depthwise', importance_type=None,
                interaction_constraints='', learning_rate=0.005, max_bin=256,
                max_cat_to_onehot=4, max_delta_step=0, max_depth=3, max_leaves=0,
                min_child_weight=5, monotone_constraints='()',
                n_estimators=900, n_jobs=0, num_parallel_tree=1, predictor='auto',
                random_state=0, reg_alpha=0)
        #modelo_xgb = xgb.XGBRegressor(early_stop_rounds = 100)
        #xgb_rand_search = RandomizedSearchCV(modelo_xgb, params_xgb, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
        #xgb_rand_search.fit(x_treino, y_treino)
        #modelo_xgb = xgb_rand_search.best_estimator_
        modelo_xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino)],verbose=False)
        end_xgb = datetime.now()

        #Criação e fit do modelo Random Forest
        start_rf = datetime.now()
        modelo_rf = RandomForestRegressor(max_depth=10, max_features='log2', min_samples_leaf=4,
                        min_samples_split=7, n_estimators=900)
        #modelo_rf = RandomForestRegressor()
        #rf_rand_search = RandomizedSearchCV(modelo_rf, params_rf, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
        #rf_rand_search.fit(x_treino, y_treino)
        #modelo_rf = rf_rand_search.best_estimator_
        modelo_rf.fit(x_treino, y_treino)
        end_rf = datetime.now()

        #Criação e fit do modelo Linear Regression
        start_lr = datetime.now()
        modelo_lr = LinearRegression()
        modelo_lr.fit(x_treino, y_treino)
        end_lr = datetime.now()


        tempo_xgb = end_xgb - start_xgb
        tempo_xgb = float(f"{str(tempo_xgb.seconds)}.{str(tempo_xgb.microseconds)}")
        tempo_rf = end_rf - start_rf
        tempo_rf = float(f"{str(tempo_rf.seconds)}.{str(tempo_rf.microseconds)}")
        tempo_lr = end_lr - start_lr
        tempo_lr = float(f"{str(tempo_lr.seconds)}.{str(tempo_lr.microseconds)}")

        # Fizemos as predições e juntamos no DataFrame
        df_teste[f"predicao_xgb_{cenario}"] = modelo_xgb.predict(x_teste)
        df_teste[f"predicao_lr_{cenario}"] = modelo_lr.predict(x_teste)
        df_teste[f"predicao_rf_{cenario}"] = modelo_rf.predict(x_teste)

        # Pesquisar os motivos de se usar apenas métricas relativas - Flávia disse que provavelmente por conta de outlier/valor distantes 
        RRSE_xgb = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"predicao_xgb_{cenario}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
        MAPE_xgb = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"predicao_xgb_{cenario}"])

        RRSE_rf = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"predicao_rf_{cenario}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
        MAPE_rf = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"predicao_rf_{cenario}"])

        RRSE_lr = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"predicao_lr_{cenario}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
        MAPE_lr = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"predicao_lr_{cenario}"])

        metrics.append([f'Teste Tempos {cenario}','RRSE', RRSE_xgb, RRSE_rf, RRSE_lr])
        metrics.append([f'Teste Tempos {cenario}','MAPE', MAPE_xgb, MAPE_rf, MAPE_lr]) 
        metrics.append([f'Teste Tempos {cenario}','Tempo', tempo_xgb, tempo_rf, tempo_lr]) 

        df = df.merge(df_teste[[f'predicao_xgb_{cenario}', f'predicao_lr_{cenario}', f'predicao_rf_{cenario}']], how="left", left_index=True, right_index=True)

[21:46:10] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[21:46:16] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[21:46:21] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not 

In [192]:
metrics


[['Primeira execução',
  'RRSE',
  0.6929011501881002,
  0.7232911739283211,
  0.9394448227378863],
 ['Primeira execução',
  'MAPE',
  0.10364107666716442,
  0.10942823282525131,
  0.15358537448478704],
 ['Primeira execução', 'Tempo', 1.482996, 2.731269, 0.3],
 ['Teste Clima',
  'RRSE',
  0.6943001942678831,
  0.7225688289648855,
  0.9160853504082752],
 ['Teste Clima',
  'MAPE',
  0.1031040485254041,
  0.10939611924780253,
  0.1519430606843786],
 ['Teste Clima', 'Tempo', 1.413068, 2.359836, 0.2999],
 ['Teste Tempos 1',
  'RRSE',
  0.7026246885613601,
  0.7116102098709544,
  0.8001398436184974],
 ['Teste Tempos 1',
  'MAPE',
  0.1045885023752492,
  0.10610819856920628,
  0.12699859239938566],
 ['Teste Tempos 1', 'Tempo', 1.423969, 3.463199, 0.1998],
 ['Teste Tempos 2',
  'RRSE',
  0.6898073285857785,
  0.6912333834480877,
  0.7435445044861322],
 ['Teste Tempos 2',
  'MAPE',
  0.10322757733328276,
  0.10386794706932669,
  0.11667826106146552],
 ['Teste Tempos 2', 'Tempo', 1.538178, 3.795

In [193]:
df = df.merge(df_teste[['tempo_viagem_1','tempo_viagem_2','tempo_viagem_3','tempo_viagem_4']], how="left", left_index=True, right_index=True)
df_teste.head()

,data_partida,tempo_viagem,dia_semana,partidaTimeStamp,tipo_dia,turno_dia,hora_dia,predicao_xgb,predicao_lr,predicao_rf,...,predicao_rf_1,predicao_xgb_2,predicao_lr_2,predicao_rf_2,predicao_xgb_3,predicao_lr_3,predicao_rf_3,predicao_xgb_4,predicao_lr_4,predicao_rf_4
6730,2019-09-30 22:06:32,1055,1,1569891992,1,4,22,1071.265137,1356.958497,1206.589859,...,1085.971464,1033.455322,1083.781300,1038.665889,1047.306152,1094.909011,1028.194178,1047.920166,1080.839335,1029.732912
6963,2019-09-30 21:50:37,1428,1,1569891037,1,4,21,1094.671509,1340.312410,1211.542185,...,1142.429527,1037.720459,1120.008162,1066.822956,1027.292603,1089.611186,1042.225117,1057.315063,1102.536670,1064.079603
7347,2019-09-30 21:45:34,986,1,1569890734,1,4,21,1094.671509,1340.311494,1211.542185,...,1170.038398,1097.488281,1286.584728,1141.870128,1093.381592,1218.850773,1119.059236,1089.997070,1181.511792,1108.916310
8001,2019-09-30 21:32:06,1086,1,1569889926,1,4,21,1094.671509,1340.309053,1211.542185,...,1135.098050,1090.693970,1259.291068,1194.726983,1088.927979,1219.926858,1138.725499,1106.074097,1181.558921,1097.153991
6729,2019-09-30 21:21:11,1287,1,1569889271,1,4,21,1094.671509,1340.307074,1211.542185,...,1147.710816,1060.024414,1143.176976,1084.639270,1147.653198,1224.469190,1177.103692,1159.932739,1203.169402,1190.593285


# Testes Intervalos de Teste_Treino

In [194]:
#Separamos treino e teste
df_intervalos = df.copy()
df_intervalos["data_partida2"] = pd.to_datetime(df_intervalos['data_partida'])

df_teste.head()

,data_partida,tempo_viagem,dia_semana,partidaTimeStamp,tipo_dia,turno_dia,hora_dia,predicao_xgb,predicao_lr,predicao_rf,...,predicao_rf_1,predicao_xgb_2,predicao_lr_2,predicao_rf_2,predicao_xgb_3,predicao_lr_3,predicao_rf_3,predicao_xgb_4,predicao_lr_4,predicao_rf_4
6730,2019-09-30 22:06:32,1055,1,1569891992,1,4,22,1071.265137,1356.958497,1206.589859,...,1085.971464,1033.455322,1083.781300,1038.665889,1047.306152,1094.909011,1028.194178,1047.920166,1080.839335,1029.732912
6963,2019-09-30 21:50:37,1428,1,1569891037,1,4,21,1094.671509,1340.312410,1211.542185,...,1142.429527,1037.720459,1120.008162,1066.822956,1027.292603,1089.611186,1042.225117,1057.315063,1102.536670,1064.079603
7347,2019-09-30 21:45:34,986,1,1569890734,1,4,21,1094.671509,1340.311494,1211.542185,...,1170.038398,1097.488281,1286.584728,1141.870128,1093.381592,1218.850773,1119.059236,1089.997070,1181.511792,1108.916310
8001,2019-09-30 21:32:06,1086,1,1569889926,1,4,21,1094.671509,1340.309053,1211.542185,...,1135.098050,1090.693970,1259.291068,1194.726983,1088.927979,1219.926858,1138.725499,1106.074097,1181.558921,1097.153991
6729,2019-09-30 21:21:11,1287,1,1569889271,1,4,21,1094.671509,1340.307074,1211.542185,...,1147.710816,1060.024414,1143.176976,1084.639270,1147.653198,1224.469190,1177.103692,1159.932739,1203.169402,1190.593285


In [195]:
data_atual = df_intervalos["data_partida2"].dt.date.min() + timedelta(days=14)
dias_treino = 14
dias_teste = 79


while data_atual < (df_intervalos["data_partida2"].dt.date.max() - timedelta(days=14)):
        df_teste = df_intervalos[df_intervalos["data_partida2"].dt.date > data_atual]
        df_treino = df_intervalos[df_intervalos["data_partida2"].dt.date <= data_atual]

        #Separamos a classe dos atributos 
        x_teste = df_teste[['dia_semana','partidaTimeStamp','tipo_dia','hora_dia']]
        y_teste = df_teste[['tempo_viagem']]
        x_treino = df_treino[['dia_semana','partidaTimeStamp','tipo_dia','hora_dia']]
        y_treino = df_treino[['tempo_viagem']]

        #Criação e fit do modelo XGB
        start_xgb = datetime.now()
        modelo_xgb = xgb.XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
                colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
                early_stop_rounds=100, early_stopping_rounds=None,
                enable_categorical=False, eval_metric=None, gamma=0.0, gpu_id=-1,
                grow_policy='depthwise', importance_type=None,
                interaction_constraints='', learning_rate=0.005, max_bin=256,
                max_cat_to_onehot=4, max_delta_step=0, max_depth=3, max_leaves=0,
                min_child_weight=5, monotone_constraints='()',
                n_estimators=900, n_jobs=0, num_parallel_tree=1, predictor='auto',
                random_state=0, reg_alpha=0)
        #modelo_xgb = xgb.XGBRegressor(early_stop_rounds = 100)
        #xgb_rand_search = RandomizedSearchCV(modelo_xgb, params_xgb, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
        #xgb_rand_search.fit(x_treino, y_treino)
        #modelo_xgb = xgb_rand_search.best_estimator_
        modelo_xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino)],verbose=False)
        end_xgb = datetime.now()

        #Criação e fit do modelo Random Forest
        start_rf = datetime.now()
        modelo_rf = RandomForestRegressor(max_depth=10, max_features='log2', min_samples_leaf=4,
                        min_samples_split=7, n_estimators=900)
        #modelo_rf = RandomForestRegressor()
        #rf_rand_search = RandomizedSearchCV(modelo_rf, params_rf, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
        #rf_rand_search.fit(x_treino, y_treino)
        #modelo_rf = rf_rand_search.best_estimator_
        modelo_rf.fit(x_treino, y_treino)
        end_rf = datetime.now()

        #Criação e fit do modelo Linear Regression
        start_lr = datetime.now()
        modelo_lr = LinearRegression()
        modelo_lr.fit(x_treino, y_treino)
        end_lr = datetime.now()


        tempo_xgb = end_xgb - start_xgb
        tempo_xgb = float(f"{str(tempo_xgb.seconds)}.{str(tempo_xgb.microseconds)}")
        tempo_rf = end_rf - start_rf
        tempo_rf = float(f"{str(tempo_rf.seconds)}.{str(tempo_rf.microseconds)}")
        tempo_lr = end_lr - start_lr
        tempo_lr = float(f"{str(tempo_lr.seconds)}.{str(tempo_lr.microseconds)}")
        
        # Fizemos as predições e juntamos no DataFrame 
        df_teste[f"predicao_xgb {dias_treino} {dias_teste}"] = modelo_xgb.predict(x_teste)
        df_teste[f"predicao_lr {dias_treino} {dias_teste}"] = modelo_lr.predict(x_teste)
        df_teste[f"predicao_rf {dias_treino} {dias_teste}"] = modelo_rf.predict(x_teste)

        # Pesquisar os motivos de se usar apenas métricas relativas - Flávia disse que provavelmente por conta de outlier/valor distantes 
        RRSE_xgb = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"predicao_xgb {dias_treino} {dias_teste}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
        MAPE_xgb = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"predicao_xgb {dias_treino} {dias_teste}"])

        RRSE_rf = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"predicao_rf {dias_treino} {dias_teste}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
        MAPE_rf = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"predicao_rf {dias_treino} {dias_teste}"])

        RRSE_lr = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"predicao_lr {dias_treino} {dias_teste}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
        MAPE_lr = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"predicao_lr {dias_treino} {dias_teste}"])

        metrics.append([f'Teste Intervalos {dias_treino} {dias_teste}','RRSE', RRSE_xgb, RRSE_rf, RRSE_lr])
        metrics.append([f'Teste Intervalos {dias_treino} {dias_teste}','MAPE', MAPE_xgb, MAPE_rf, MAPE_lr]) 
        metrics.append([f'Teste Intervalos {dias_treino} {dias_teste}','Tempo', tempo_xgb, tempo_rf, tempo_lr])

        df = df.merge(df_teste[[f'predicao_xgb {dias_treino} {dias_teste}',f'predicao_rf {dias_treino} {dias_teste}',f'predicao_lr {dias_treino} {dias_teste}']], how="left", left_index=True, right_index=True)
        

        dias_teste -= 14
        dias_treino += 14

        data_atual += timedelta(days=14)



[21:46:35] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\2225869443.py:43: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  modelo_rf.fit(x_treino, y_treino)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\2225869443.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f"predicao_xgb {dias_treino} {dias_teste}"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\2225869443.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

[21:46:37] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\2225869443.py:43: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  modelo_rf.fit(x_treino, y_treino)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\2225869443.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f"predicao_xgb {dias_treino} {dias_teste}"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\2225869443.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

[21:46:39] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\2225869443.py:43: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  modelo_rf.fit(x_treino, y_treino)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\2225869443.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f"predicao_xgb {dias_treino} {dias_teste}"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\2225869443.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

[21:46:43] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\2225869443.py:43: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  modelo_rf.fit(x_treino, y_treino)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\2225869443.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f"predicao_xgb {dias_treino} {dias_teste}"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\2225869443.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

[21:46:47] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\2225869443.py:43: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  modelo_rf.fit(x_treino, y_treino)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\2225869443.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f"predicao_xgb {dias_treino} {dias_teste}"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\2225869443.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [196]:
metrics = pd.DataFrame(metrics, columns=['Descrição','Métrica', 'XGBoosting','Random Forest','Linear Regression'])
metrics.to_csv(f'metricas{rota}.csv',index=False)
metrics

,Descrição,Métrica,XGBoosting,Random Forest,Linear Regression
0,Primeira execução,RRSE,0.692901,0.723291,0.939445
1,Primeira execução,MAPE,0.103641,0.109428,0.153585
2,Primeira execução,Tempo,1.482996,2.731269,0.300000
3,Teste Clima,RRSE,0.694300,0.722569,0.916085
4,Teste Clima,MAPE,0.103104,0.109396,0.151943
5,Teste Clima,Tempo,1.413068,2.359836,0.299900
6,Teste Tempos 1,RRSE,0.702625,0.711610,0.800140
7,Teste Tempos 1,MAPE,0.104589,0.106108,0.126999
8,Teste Tempos 1,Tempo,1.423969,3.463199,0.199800
9,Teste Tempos 2,RRSE,0.689807,0.691233,0.743545


In [197]:
df.to_csv(f'previsão_{rota}.csv',index=False)

In [198]:
df.shape

(9314, 46)